In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization, Dropout  , Flatten, Dense

In [2]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation = 'relu', padding= 'same', input_shape = (32,32,3)))
model.add(Conv2D(32, (3,3), activation = 'relu', padding= 'same'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(64, (3,3), activation = 'relu', padding = 'same'))
model.add(Conv2D(64, (3,3), activation = 'relu', padding = 'same'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(128, (3,3), activation = 'relu', padding = 'same'))
model.add(Conv2D(128, (3,3), activation = 'relu', padding = 'same'))
model.add(MaxPool2D((2,2)))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(10, activation = 'softmax'))

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 64)         0         
 2D)                                                    

In [4]:
model.compile(optimizer= 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [5]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

In [6]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)

In [7]:
model.fit(x_train, y_train, epochs = 5, batch_size= 32, validation_data= (x_test, y_test))

Epoch 1/5
1563/1563 [==============================] - 165s 105ms/step - loss: 1.6665 - accuracy: 0.4020 - val_loss: 3.0350 - val_accuracy: 0.2878
Epoch 2/5
1563/1563 [==============================] - 167s 107ms/step - loss: 1.3040 - accuracy: 0.5387 - val_loss: 2.8405 - val_accuracy: 0.5672
Epoch 3/5
1563/1563 [==============================] - 177s 113ms/step - loss: 1.0992 - accuracy: 0.6183 - val_loss: 1.3498 - val_accuracy: 0.6215
Epoch 4/5
1563/1563 [==============================] - 169s 108ms/step - loss: 0.9837 - accuracy: 0.6579 - val_loss: 1.6635 - val_accuracy: 0.6106
Epoch 5/5
1563/1563 [==============================] - 186s 119ms/step - loss: 0.9019 - accuracy: 0.6900 - val_loss: 1.4619 - val_accuracy: 0.6123


## Let us build a simple VAE

In [77]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Lambda, Input
from tensorflow.keras.models import Model
from keras.losses import binary_crossentropy
from keras.optimizers import Adam
from keras import metrics
from keras import losses

In [78]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [79]:
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [80]:
from tensorflow.keras.layers import Reshape, Input
import numpy as np

In [81]:
input = Input(shape = (32,32,3))
l_1 = Conv2D(32, (3,3), padding = 'same')(input)
l_2 = Conv2D(64, (3,3), padding = 'same')(l_1)
l_3 = Flatten()(l_2)
l_4 = Dense(128, activation = 'relu')(l_3)
z_mean = Dense(2)(l_4)
z_log_var =Dense(2)(l_4)
z= Lambda(sampling, (2,))([z_mean, z_log_var])
encoder = Model(input, [z_mean, z_log_var])

In [82]:
encoder.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_20 (Conv2D)             (None, 32, 32, 32)   896         ['input_1[0][0]']                
                                                                                                  
 conv2d_21 (Conv2D)             (None, 32, 32, 64)   18496       ['conv2d_20[0][0]']              
                                                                                                  
 flatten_6 (Flatten)            (None, 65536)        0           ['conv2d_21[0][0]']              
                                                                                            

In [83]:
decoder_input = Input(shape = (2,))
d_1 = Dense(128, activation = 'relu', input_shape = (2,))(decoder_input)
d_2 = Dense(32*32*3, activation = 'sigmoid')(d_1)
d_3 = Reshape((32,32,3))(d_2)
decoder = Model(decoder_input, d_3)

In [84]:
decoder.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense_30 (Dense)            (None, 128)               384       
                                                                 
 dense_31 (Dense)            (None, 3072)              396288    
                                                                 
 reshape_7 (Reshape)         (None, 32, 32, 3)         0         
                                                                 
Total params: 396,672
Trainable params: 396,672
Non-trainable params: 0
_________________________________________________________________


In [85]:
output_combined = decoder(encoder(input)[1])
vae = Model(input, output_combined)
vae.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 model_5 (Functional)        [(None, 2),               8408644   
                              (None, 2)]                         
                                                                 
 model_6 (Functional)        (None, 32, 32, 3)         396672    
                                                                 
Total params: 8,805,316
Trainable params: 8,805,316
Non-trainable params: 0
_________________________________________________________________


In [86]:
def vae_loss(x, x_decoded_mean):
    # Reconstruction loss
    xent_loss = binary_crossentropy(K.flatten(x), K.flatten(x_decoded_mean))

    # KL divergence loss
    kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)

    # Reshape xent_loss to be compatible with kl_loss
    xent_loss = K.reshape(xent_loss, shape=(-1, 1))

    return xent_loss + kl_loss

In [87]:
vae.compile(optimizer=Adam(), loss = vae_loss)
vae.fit(x_train, x_train, epochs = 5, validation_data=(x_test, x_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - ETA: 0s - loss: nan

C:\Users\amade\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training_v1.py:2332: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


50000/50000 [==============================] - 232s 5ms/sample - loss: nan - val_loss: nan
Epoch 2/5
50000/50000 [==============================] - 245s 5ms/sample - loss: nan - val_loss: nan
Epoch 3/5
50000/50000 [==============================] - 240s 5ms/sample - loss: nan - val_loss: nan
Epoch 4/5
50000/50000 [==============================] - 247s 5ms/sample - loss: nan - val_loss: nan
Epoch 5/5
50000/50000 [==============================] - 212s 4ms/sample - loss: nan - val_loss: nan
